In [5]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math

In [2]:
detector = HandDetector(detectionCon=0.7, maxHands=2)

In [48]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

while True:
    success, img = cap.read()
    if not success:
        break

    hands, img = detector.findHands(img)

    if len(hands) == 2:
        hand1, hand2 = hands
        if hand1["center"][0] < hand2["center"][0]:
            left_hand = hand2
            right_hand = hand1
        else:
            left_hand = hand1
            right_hand = hand2

        fingers_left = detector.fingersUp(left_hand)
        freq_band = "none"
        if fingers_left == [0, 0, 0, 0, 0]:
            freq_band = "bass"
        elif fingers_left == [0, 1, 0, 0, 0]:
            freq_band = "mid"
        elif fingers_left == [0, 0, 0, 0, 1]:
            freq_band = "treble"
        elif fingers_left == [1, 1, 1, 1, 1]:
            freq_band = "all"

        if freq_band in ["bass", "mid", "treble"]:
            distance, info, img = detector.findDistance(
                right_hand["lmList"][8][:-1], right_hand["lmList"][4][:-1], img)
            gain = max(0, min(100, int(distance * 0.5)))
        elif freq_band == "all":
            wrist = np.array(right_hand['lmList'][0])
            middle_tip = np.array(right_hand['lmList'][12])

            dx = middle_tip[0] - wrist[0]
            dy = middle_tip[1] - wrist[1]
            angle = -math.degrees(math.atan2(dy, dx)) - 90
            angle_clamped = max(-50, min(50, angle))
            gain = int(angle_clamped/50*100)
        else:
            gain = 0

        cv2.putText(img, f"Freq Band: {freq_band}", (10, 70),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3,)
        cv2.putText(img, f"Gain (Distance): {gain}", (10, 110),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3,)

    cv2.imshow("Hand Gesture Equalizer Control", img)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()